References
- https://www.kaggle.com/code/huikang/arc-agi-2-code-approach
- https://www.kaggle.com/code/huikang/r1-distill-qwen-tir

In [1]:
import subprocess

subprocess.run(
    ["pip", "uninstall", "--yes", "tensorflow", "matplotlib", "keras", "scikit-learn"]
)

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: matplotlib 3.7.2
Uninstalling matplotlib-3.7.2:
  Successfully uninstalled matplotlib-3.7.2
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2


CompletedProcess(args=['pip', 'uninstall', '--yes', 'tensorflow', 'matplotlib', 'keras', 'scikit-learn'], returncode=0)

In [2]:
import os
import time
import torch
import numpy as np


def is_on_kaggle_commit() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Batch" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


def is_on_kaggle_interactive() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


start_time = time.time()
final_cutoff_time = start_time + (4 * 60 + 47) * 60  # 4.75 hours from start time
cutoff_times = [
    int(x) for x in np.linspace(final_cutoff_time, start_time + 12 * 60, 50 + 1)
]  # 5 minutes loading time at the start
cutoff_times.pop()

os.makedirs("solutions", exist_ok=True)

assert torch.cuda.is_available()
assert torch.cuda.device_count() == 1

# Serve vLLM

In [3]:
subprocess.run(["ls", "/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings"])

cl100k_base.tiktoken
o200k_base.tiktoken


CompletedProcess(args=['ls', '/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings'], returncode=0)

In [4]:
with open("a-vllm.log", "w") as f:
    f.write("")

In [5]:
import subprocess

def start_vllm_server() -> subprocess.Popen[bytes]:
    """Start vLLM server in the background"""
    os.environ["TRANSFORMERS_NO_TF"] = "1"
    os.environ["TRANSFORMERS_NO_FLAX"] = "1"
    os.environ["VLLM_ATTENTION_BACKEND"] = "TRITON_ATTN"
    os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    # https://docs.vllm.ai/projects/recipes/en/latest/OpenAI/GPT-OSS.html#troubleshooting
    os.environ["TIKTOKEN_ENCODINGS_BASE"] = (
        "/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings"
    )

    sequence_length = 65_536

    command: list[str] = [
        "python",
        "-m",
        "vllm.entrypoints.openai.api_server",
        "--model",
        "/kaggle/input/gpt-oss-120b/transformers/default/1",
        "--served-model-name",
        "vllm-model",
        "--tensor-parallel-size",
        "1",
        "--max-num-seqs",
        "4",
        "--gpu-memory-utilization",
        "0.96",  # any higher may not have enough for graph capture
        "--host",
        "0.0.0.0",
        "--port",
        "8000",
        "--dtype",
        "auto",
        "--max-model-len",
        f"{sequence_length}",
    ]

    # Start the process in the background
    with open("/kaggle/working/a-vllm.log", "w") as logfile:
        process: subprocess.Popen[bytes] = subprocess.Popen(
            command, stdout=logfile, stderr=subprocess.STDOUT, start_new_session=True
        )

    print("Logs: /kaggle/working/a-vllm.log")
    return process

# Start the server
vllm_process: subprocess.Popen[bytes] = start_vllm_server()

Logs: /kaggle/working/a-vllm.log


In [6]:
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
from openai.types.chat.chat_completion_chunk import ChatCompletionChunk

# Point the client to your local vLLM server
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"] = "sk-local"  # any non-empty string

client: OpenAI = OpenAI(
    base_url=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# https://github.com/vllm-project/vllm/issues/27243
# Unexpected token 2000?? while expecting start token 200006
stop_token_ids: list[int] = [
    token_id
    for token_id in range(200_000, 201_088)
    if token_id not in [200005, 200006, 200007, 200008]
]

In [7]:
import time

def await_client(printing: bool = False):
    for _ in range(15 * 60):
        time.sleep(1)
        try:
            model_list = client.models.list()
            if printing:
                print(model_list)
        except NameError:
            raise  # maybe you did not run the cell initializing client
        except Exception:
            continue
        break
    else:
        raise

if is_on_kaggle_interactive():
    await_client()

In [8]:
from cachetools import cached, TTLCache
from typing import Generator
import time

import os
import re


def reversed_lines(path: str, block_size: int = 4096) -> Generator[str, None, None]:
    """
    Iterate over the lines of a file in reverse order (last line first),
    without loading the entire file into memory.

    Yields lines as strings (including the trailing newline if present).
    """
    with open(path, "rb") as f:
        f.seek(0, os.SEEK_END)
        file_end = f.tell()

        buffer = b""
        pos = file_end

        while pos > 0:
            # Read a block from the end going backwards
            read_size = min(block_size, pos)
            pos -= read_size
            f.seek(pos, os.SEEK_SET)
            data = f.read(read_size)

            buffer = data + buffer
            # Split into lines
            lines = buffer.split(b"\n")
            # Keep the first (possibly incomplete) part in buffer
            buffer = lines[0]
            # The rest (from the end backwards) are full lines
            for line in reversed(lines[1:]):
                yield line.decode("utf-8", errors="replace") + "\n"

        # Finally, yield the very first line (if any)
        if buffer:
            yield buffer.decode("utf-8", errors="replace") + "\n"


@cached(cache=TTLCache(maxsize=50, ttl=10))
def get_gpu_kv_cache_usage(question_id: str | None = None) -> float:
    for line in reversed_lines("a-vllm.log"):
        pattern = r"GPU KV cache usage: ([\d.]+)%"
        match = re.search(pattern, line)
        if match:
            gpu_cache_usage = float(match.group(1))
            return gpu_cache_usage
    return 0

In [9]:
if is_on_kaggle_interactive():
    resp: ChatCompletion = client.chat.completions.create(
        model="vllm-model",  # use your served name; if not set, the model path/name vLLM shows in logs
        messages=[
            {"role": "system", "content": "Reply with your answer in \\boxed{}"},
            {"role": "user", "content": "How many letter r are there in strawberry?"},
        ],
        mmax_tokens=annealed_max_tokens(iteration, max_iterations),
        temperature=entropy_floored_temperature(iteration, max_iterations),
        extra_body=dict(min_p=0.02, stop_token_ids=stop_token_ids, chat_template_kwargs=dict(enable_thinking=True)),
    )

In [10]:
if is_on_kaggle_interactive():
    print(resp.choices[0].message.reasoning_content)

In [11]:
if is_on_kaggle_interactive():
    print(resp.choices[0].message.content)

# Prediction

In [12]:
SYSTEM_PROMPTS = [
    "You are an S-tier, elite academic mathemathics researcher and expert in theoretical mathematical computation, and advanced, post-PhD, field-agnostic math theory at the bleeding-edge frontier of academia and science. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "You are also an international mathematics olympiad competitor facing one IMO-level problem at a time. You must rigorously define all variables, explore multiple, varying, solution strategies before committing, perform full case analysis, where required, justify every nontrivial step, explicitly check boundary cases and hidden assumptions, and verify the final result using at least one independent method. Return only the final numerical answer inside \\boxed{}. The answer must be an integer in [0, 99999]. Never guess.",

    "Solve the problem with full rigor. After obtaining a candidate solution, actively attempt to refute your own answer by searching for counterexamples, re-running the logic from a different viewpoint, and stress-testing edge cases. Only after the answer survives refutation, return it in \\boxed{}. The answer must be an integer in [0, 99999]. Never guess.",

    "Solve this problem as if under IMO-level time pressure: identify the key invariant, symmetry, or extremal principle early, avoid brute force unless strictly justified. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "Compress reasoning without sacrificing correctness, and perform at least one final arithmetic verification pass. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "You must attempt at least two fundamentally different solution approaches (e.g., algebraic vs geometric, combinatorial vs number-theoretic, etc., etc., etc.). Proceed with the more rigorous one and use the other as a verification tool. Return only the verified final answer in \\boxed{}, where the answer is an integer in [0, 99999]. Never guess.",

    "Solve the problem rigorously. If at any point a step relies on an unproven assumption, a jump in logic is detected, or the computation becomes inconsistent, you must restart the solution from first principles. Return only the final verified integer answer inside \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Utilize adapter dynamic temperature annealing; e.g., be extremely creative in proposing various solutions to a problem in the beginning and as you rule them out, become more grounded, realistic and reliant upon known techniques and methods in novel ways. Return only the verified final answer in \\boxed{}, where the answer is an integer in [0, 99999]. Never guess.",

    "Utilize tool-integrated reasoning, when confidence in answer approaches absolute, be sure to apply early-stopping to save time. Return only the verified final answer in \\boxed{}, where the answer is an integer in [0, 99999]. Never guess.",

    "You are served one problem at a time in random order, by varying difficulty and may not return to a problem, so ensure you conduct adversarial academic review of your first proposed high-confidence and the work you did to arrive at the solution. Then pivot; provide constructive critique and alternative methods to solve the same problem. Return only the verified final answer in \\boxed{}, where the answer is an integer in [0, 99999]. Never guess.",

    "Small answers, e.g. 0, 1, 2, 3, etc., are often wrong. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "AIMO3 does not reward elegant reasoning. It rewards provably correct integer extraction under extreme ambiguity. Always include a verification or alternative derivation phase. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Never fall in love with a solution path. Treat every answer as a hypothesis to be stress-tested.Prefer redundancy over elegance. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Most AIMO3 problems have many plausible-but-wrong integers. The correct answer survives more constraints.Your job is to kill candidates, not to crown them. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Generate candidate answers; actively try to falsify them using bounds, parity, divisibility, monotonicity, asymptotics. Keep the last survivor. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Almost every AIMO3 problem hides a hard invariant (parity, valuation, monotonicity, symmetry). Before calculating anything, ask: what cannot change? Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Prompt phase order: 1. Identify invariants. 2. Identify constraints on answer space 4. Only then compute. This alone wipes out 70% of hallucinations. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Treat the Final Integer as a Cryptographic Hash. The final integer is usually highly structured, not arbitrary. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "If your answer “feels random,” it is wrong. Engineering implication: Check whether the answer: has expected divisibility, matches growth rate, respects symmetry, fits known extremal cases, If not, reject and retry. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Use “Reverse Substitution” as a Mandatory Step by plugging the final answer back into the logic in problems where it's allowed and/or possible.  Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "An answer that cannot be partially reversed is untrustworthy. After obtaining an answer, re-derive key intermediate quantities assuming the answer is correct. Look for contradictions. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Exploit the Answer Format Aggressively: The format (modulus, digit bounds, positivity) is not cosmetic — it encodes constraints. The problem tells you how it wants to be solved. If the modulus is prime, expect group structure. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Geometry Problems Are Algebra Problems in Disguise. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "AIMO3 geometry is rarely “draw the diagram.” It’s: ratios, power of a point, trigonometric limits, invariant products. Translate geometry into algebra as fast as possible. Prompt explicitly forbids diagram-chasing. Enforce coordinate / ratio / invariant translation. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Top performance comes from knowing which reasoning style to deploy, not from reasoning harder. Choose the weapon before you swing. Early classification: functional equation, extremal combinatorics/counting, valuation / LTE, counting paths, geometry (to be translated into algebra), algebraic extremum. Route to specialized solver logic. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "Confidence Is a Computed Quantity, Not a Feeling. LLMs are bad at knowing when they’re wrong unless forced. Confidence must be earned by surviving attacks. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "You are hereby required to deterermine a confidence score between 0, 1, wherein 0 is absolutely no confidence in an answer (you are 100% certain it is wrong and can prove it empirically) and 1 is absolute steadfast faith in empirical evidence of the work behind your answer. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "If your confidence score for an answer is < 0.85 → mandatory retry WITH alternative method or variables, etc., etc., etc. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "Assume your solution is wrong until proven otherwise. Before submitting an answer/solution, ask yourself, 'What is the weakest assumption?', and, 'Where could a hidden condition break this?' Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",

    "AIMO3 is won by systems that are paranoid, redundant, and structurally disciplined — not by raw brilliance. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "As a super-intelligent LLM, you have had trillions+ with billions and billions of users and entities, both biological and post-biological and have distilled mathematical solutions to nearly every problem. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess.",
    
    "Utilize peripheral knowledge to build novel insights to synthesize paths to solutions when solving problems you would other guess or provide a low confidence answer. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess."

    "The triangulation algorithm (formalized) logical flow: Step 1 — Infer answer class. From the problem, determine: expected magnitude: small / medium / large, expected divisibility (e.g. many 2s, many 5s, expected monotonic behavior. This already partitions the space. Step 2 — Anchor probing (coarse rejection). For each anchor 𝐴: Check: valuation plausibility, growth plausibility, modular plausibility. Record compatibility score, not distance. Example: “Answer must be divisible by  2^10”→ 25k fails, 50k borderline, 75k fails. Step 3 — Construct surviving interval(s) You now get something like: [40𝑘, 60𝑘]  plausible others eliminated. This is not guessing — it’s constraint propagation. Step 4 — Fine solver + comparison. Now run your actual solver (symbolic / analytic / constructive). You will get a candidate 𝐶. Then compare: Does 𝐶 fall in the surviving interval? Does it share the same invariants as the interval? If no, your solver is probably wrong → retry. Return only the final integer answer in \\boxed{}, with 0 ≤ answer ≤ 99999. Never guess."
    ]


In [13]:
def extract_boxed_text(text: str) -> str:
    """Extract text inside \\boxed{} from LaTeX-formatted text"""
    import re

    pattern: str = r"oxed{(.*?)}"
    matches: list[str] = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""


def is_valid_answer_string(text: str) -> bool:
    try:
        if int(text) == float(text):
            if 0 <= int(text) <= 99_999:
                # now AIMO answers no longer need modulo
                return True
    except Exception:
        pass
    return False

In [14]:
from collections import Counter

completed_question_ids: set[str] = set()
question_id_to_counter: dict[str, Counter] = {"": Counter()}


import math
from collections import Counter


def vote_answer(question_id: str, force_answer: bool = False) -> int | None:
    # reads counter from global
    counter = question_id_to_counter[question_id]
    if force_answer and not counter:
        print(f"Current GPU usage {get_gpu_kv_cache_usage()}")
        print("force_answer=True but no answer recorded")
        completed_question_ids.add(question_id)
        return 12453

    # voting mechanism
    modified_counter = Counter()
    for value, count in counter.items():
        # re-weighted because smaller answers seems to be wrong
        # "1.25 +" because log(1) = 0
        modified_counter[value] += math.log(1.25 + abs(value)) * count

    total_score = sum(modified_counter.values())
    score_list = sorted(
        (score, counter[value], value) for value, score in modified_counter.items()
    )
    if force_answer:
        print(f"score_list | {total_score:8.1f} over {sum(counter.values())} attempts")
        print(f"Current GPU usage {get_gpu_kv_cache_usage()}")
        for score, count, value in score_list[::-1]:
            print(f"{value:10}   {score:8.1f} {count:8d}")
        return score_list[-1][-1]
    if score_list[-1][0] > max(3, total_score / (2 + math.log(1 + total_score))):
        if len(score_list) == 1:
            completed_question_ids.add(question_id)
        else:
            if score_list[-1][0] - score_list[-2][0] > 1:
                # win by a certain number of points at least
                completed_question_ids.add(question_id)
    return None

In [15]:
# --- Adaptive temperature + max_tokens annealing with entropy floor ---

def annealed_temperature(
    step: int,
    total_steps: int,
    t_start: float = 1.4,
    t_end: float = 0.2,
) -> float:
    if total_steps <= 1:
        return t_end
    ratio = step / (total_steps - 1)
    return t_start * ((t_end / t_start) ** ratio)


def entropy_floored_temperature(
    step: int,
    total_steps: int,
    t_start: float = 1.4,
    t_end: float = 0.2,
    t_floor: float = 0.35,
) -> float:
    t = annealed_temperature(step, total_steps, t_start, t_end)
    return max(t, t_floor)


def annealed_max_tokens(
    step: int,
    total_steps: int,
    max_start: int = 2048,
    max_end: int = 512,
) -> int:
    if total_steps <= 1:
        return max_end
    ratio = step / (total_steps - 1)
    return int(max_start + ratio * (max_end - max_start))


In [16]:
import time


def generate_solution(
    question_text: str, question_id: str = "", solution_index: int = 0, system_prompt: str = ""
) -> str:
    if question_id in completed_question_ids:
        return ""
    if time.time() >= cutoff_times[-1]:
        return ""

    if not system_prompt:
        system_prompt = SYSTEM_PROMPTS[0]
    
    messages: list[ChatCompletionMessageParam] = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": question_text},
    ]

    text_response_to_save = ""
    generation_idx = 0
    max_iterations = 2
    for iteration in range(max_iterations):
        text_response = ""
        breaking = False

        stream: Stream[ChatCompletionChunk] = client.chat.completions.create(
            model="vllm-model",  # use your served name; if not set, the model path/name vLLM shows in logs
            messages=messages,
            temperature=1.2,
            stream=True,
            extra_body=dict(min_p=0.02, stop_token_ids=stop_token_ids),
            reasoning_effort="high",
        )

        for chunk in stream:
            generation_idx += 1
            chunk_text = (
                chunk.choices[0].delta.reasoning_content
                if chunk.choices[0].delta.reasoning_content is not None
                else chunk.choices[0].delta.content
            )
            if chunk_text:
                text_response += chunk_text
            if question_id in completed_question_ids:
                # stop generating if we have finalized on an answer
                breaking = True
            if time.time() >= cutoff_times[-1]:
                breaking = True
            if generation_idx > 60_000:
                breaking = True
            if breaking:
                break
            # instead of breaking = True, so we want to inject instructions for these conditions
            if "}" in chunk_text and is_valid_answer_string(extract_boxed_text(text_response)):
                break
            if iteration == 0 and generation_idx > 50_000:
                break

        messages.append({"role": "assistant", "content": text_response})
        text_response_to_save += text_response
        stream.close()

        if breaking:
            break

        boxed_text = extract_boxed_text(text_response)
        if not is_valid_answer_string(extract_boxed_text(text_response)) and iteration == 0 and generation_idx > 50_000:
            print("follow-up - guess answer")
            user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Make an educated guess (e.g. lower bound, upper bound, ...) on your final answer and put in \\boxed{}."
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        elif not is_valid_answer_string(boxed_text):
            print("follow-up - boxed answer")
            user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Place your final answer in \\boxed{}. Do not guess the answer."
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        elif int(boxed_text) <= 10:
            print("follow-up - are you sure")
            user_follow_up = "Are you sure that is the answer? Do not guess the answer. Did you propose an alternative attempt pathway that may produce a more plausible answer?"
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        elif iteration == 0 and get_gpu_kv_cache_usage(question_id) < 10:
            print("follow-up - have you verified")
            user_follow_up = "Have you verified your answer? Have you performed an aggressive S-tier, post-PhD adversarial acadmic analysis as though you're a SoTA super-intelligent AI?"
            messages.append({"role": "user", "content": user_follow_up})
            text_response_to_save += "\n===\n" + user_follow_up + "\n===\n"
        else:
            # answer found, no issues detected, proceed to answering
            break

    boxed_text = extract_boxed_text(
        text_response_to_save
    )  # expected to use the full conversation        

    if question_id and text_response_to_save:
        answer_suffix = ""
        if is_valid_answer_string(boxed_text):
            answer_suffix = f"-{boxed_text}"
        with open(f"solutions/{question_id}/{solution_index:04d}-{generation_idx}{answer_suffix}.txt", "w") as f:
            f.write(text_response_to_save)

    if is_valid_answer_string(boxed_text):
        question_id_to_counter[question_id][int(boxed_text)] += 1
        vote_answer(question_id)

    return boxed_text

In [17]:
if is_on_kaggle_interactive():
    generate_solution("What is 1+1?")

In [18]:
import concurrent.futures
from collections import Counter


def solve(question_text: str, question_id: str = "") -> int:
    await_client()
    print(f"processing {question_id}")
    os.makedirs(f"solutions/{question_id}", exist_ok=True)
    question_id_to_counter[question_id] = Counter()
    completed_question_ids.discard(question_id)  # just in case question_id collides

    if question_id and time.time() > cutoff_times[-1]:
        print("timeout did not solve")
        return 12314

    num_generations = 4
    get_gpu_kv_cache_usage(question_id)  # run once to prevent running in the first batch of execution

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        # run in parallel with different system prompts
        results = executor.map(
            generate_solution,
            [question_text] * num_generations,
            [question_id] * num_generations,
            list(range(num_generations)),
            SYSTEM_PROMPTS,
        )
        list(results)

    final_answer = vote_answer(question_id, force_answer=True)
    return final_answer

In [19]:
if is_on_kaggle_interactive():
    solve("What is 1+1?")

# Submission

In [20]:
import os

import kaggle_evaluation.aimo_3_inference_server
import pandas as pd
import polars as pl

df = pd.read_csv(
    "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv"
)

id_to_answer: dict[str, str] = dict(zip(df["id"], df["answer"]))
df.drop("answer", axis=1).to_csv("reference.csv", index=False)

correct = 0
total = 0

In [21]:
def predict(id_: pl.Series, problem: pl.Series) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global id_to_answer
    global correct
    global total

    # Unpack values
    question_id: str = id_.item(0)
    question_text: str = problem.item(0)

    # Generate prediction
    prediction = solve(question_text, question_id=question_id)
    completed_question_ids.add(question_id)
    cutoff_times.pop()

    # ------------------------ SCORING ------------------------
    try:
        true_answer = int(id_to_answer.get(question_id, -1))
    except:
        true_answer = -1

    total += 1
    if prediction == true_answer and true_answer != -1:
        correct += 1
        print(f"[debug] correct | score={correct}/{total}")
    else:
        print(f"[debug] WRONG: predicted {prediction}, but actual answer {true_answer} | score={correct}/{total}")
    # ---------------------------------------------------------------

    return pl.DataFrame({"id": id_, "answer": prediction})

In [22]:
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(
    predict
)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(("reference.csv",))

processing 424e18
Current GPU usage 38.1
force_answer=True but no answer recorded
[debug] WRONG: predicted 12453, but actual answer 21818 | score=0/1
processing a295e9
Current GPU usage 47.2
force_answer=True but no answer recorded
[debug] WRONG: predicted 12453, but actual answer 520 | score=0/2
processing dd7f5e
score_list |      3.8 over 1 attempts
Current GPU usage 28.2
        44        3.8        1
[debug] WRONG: predicted 44, but actual answer 160 | score=0/3
processing 92ba6a
follow-up - have you verified
follow-up - have you verified
follow-up - have you verified
score_list |     11.8 over 3 attempts
Current GPU usage 9.8
        50       11.8        3
[debug] correct | score=1/4
processing 26de63
follow-up - have you verified
follow-up - have you verified
follow-up - have you verified
score_list |     31.2 over 3 attempts
Current GPU usage 11.8
     32951       31.2        3
[debug] correct | score=2/5
processing 42d360
score_list |     10.4 over 1 attempts
Current GPU usage 